Датасет для данной задачи = [датасет с kaggle](https://www.kaggle.com/datasets/blackmoon/russian-language-toxic-comments) + [датасет с huggingface](https://huggingface.co/datasets/AlexSham/Toxic_Russian_Comments), содержащие токсичные комментарии

In [82]:
!pip install datasets

In [83]:
from datasets import load_dataset

In [84]:
import pandas as pd
huggingface_data = load_dataset("AlexSham/Toxic_Russian_Comments")
kaggle_df =  pd.read_csv("kaggle_dataset.csv")

In [85]:
train_df = pd.DataFrame(huggingface_data['train'])
test_df = pd.DataFrame(huggingface_data['test'])

huggingface_df = pd.concat([train_df, test_df], ignore_index=True)

In [86]:
huggingface_df.dtypes

,0
text,object
label,int64


In [87]:
kaggle_df.dtypes

,0
comment,object
toxic,float64


In [88]:
huggingface_df.shape

(248290, 2)

In [89]:
huggingface_df.head(5)

,text,label
0,"видимо в разных регионах называют по разному ,...",0
1,"понятно что это нарушение правил, писать капсл...",1
2,"какие классные, жизненные стихи....",0
3,а и правда-когда его запретили?...,0
4,в соленой воде вирусы живут .ученые изучали со...,0


In [90]:
huggingface_df["label"].value_counts()

,count
label,
0,203685
1,44605


In [91]:
kaggle_df.shape

(14412, 2)

In [92]:
kaggle_df.head(5)

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [93]:
kaggle_df["toxic"]=kaggle_df["toxic"].apply(int)

In [94]:
kaggle_df.dtypes


,0
comment,object
toxic,int64


In [95]:
huggingface_df.dtypes

,0
text,object
label,int64


In [96]:
kaggle_df.head(5)

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1
1,"Хохлы, это отдушина затюканого россиянина, мол...",1
2,Собаке - собачья смерть\n,1
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1


In [97]:
kaggle_df["toxic"].value_counts()

,count
toxic,
0,9586
1,4826


In [98]:
kaggle_df.rename(columns={"comment":"text","toxic":"label"},inplace=True)

In [99]:
kaggle_df.head(5)

,text,label
0,"Верблюдов-то за что? Дебилы, бл...\n",1
1,"Хохлы, это отдушина затюканого россиянина, мол...",1
2,Собаке - собачья смерть\n,1
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1


In [100]:
huggingface_df.rename(columns={"comment":"text","toxic":"label"},inplace=True)

In [101]:
huggingface_df.head(5)

,text,label
0,"видимо в разных регионах называют по разному ,...",0
1,"понятно что это нарушение правил, писать капсл...",1
2,"какие классные, жизненные стихи....",0
3,а и правда-когда его запретили?...,0
4,в соленой воде вирусы живут .ученые изучали со...,0


In [102]:
merged_df=pd.concat([kaggle_df,huggingface_df],ignore_index=True)

In [103]:
merged_df.shape

(262702, 2)

**Слияние датасетов прошло успешно
Теперь нужно обработать текст**

In [104]:
import nltk
import string
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [105]:
#разбиваем текст на токены
sentence_example = merged_df.iloc[1]['text']
example_tokens=word_tokenize(sentence_example,language='russian')
#удаляем знаки пунктуации
example_tokens_without_punctuation=[i for i in example_tokens if i not in string.punctuation]
#удаляем стоп слова
russian_stop_words=stopwords.words('russian')
#удаляем стоп слова
example_tokens_without_stopwords_and_punctuation=[i for i in example_tokens_without_punctuation if i not in russian_stop_words]
#к оставшимся токенам применяем стемминг
snowball=SnowballStemmer(language='russian')
example_stemmed_tokens=[snowball.stem(i) for i in example_tokens_without_stopwords_and_punctuation]

In [106]:
print(f"Исходный текст: {sentence_example}")
print("-----------------")
print(f"Токены: {example_tokens}")
print("-----------------")
print(f"Токены без пунктуации: {example_tokens_without_punctuation}")
print("-----------------")
print(f"Токены без пунктуации и стоп слов: {example_tokens_without_stopwords_and_punctuation}")
print("-----------------")
print(f"Токены после стемминга: {example_stemmed_tokens}")
print("-----------------")

Исходный текст: Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.

-----------------
Токены: ['Хохлы', ',', 'это', 'отдушина', 'затюканого', 'россиянина', ',', 'мол', ',', 'вон', ',', 'а', 'у', 'хохлов', 'еще', 'хуже', '.', 'Если', 'бы', 'хохлов', 'не', 'было', ',', 'кисель', 'их', 'бы', 'придумал', '.']
-----------------
Токены без пунктуации: ['Хохлы', 'это', 'отдушина', 'затюканого', 'россиянина', 'мол', 'вон', 'а', 'у', 'хохлов', 'еще', 'хуже', 'Если', 'бы', 'хохлов', 'не', 'было', 'кисель', 'их', 'бы', 'придумал']
-----------------
Токены без пунктуации и стоп слов: ['Хохлы', 'это', 'отдушина', 'затюканого', 'россиянина', 'мол', 'вон', 'хохлов', 'хуже', 'Если', 'хохлов', 'кисель', 'придумал']
-----------------
Токены после стемминга: ['хохл', 'эт', 'отдушин', 'затюкан', 'россиянин', 'мол', 'вон', 'хохл', 'хуж', 'есл', 'хохл', 'кисел', 'придума']
-----------------


**Теперь обработаем текст, но не для одного примера, а для всего датасета**

In [107]:
snowball=SnowballStemmer(language='russian')
russian_stop_words=stopwords.words('russian')

In [108]:
def tokenize_sentence(sentence: str, remove_stopwords: bool=True):
  tokens=word_tokenize(sentence,language='russian')
  tokens=[i for i in tokens if i not in string.punctuation]
  if remove_stopwords:
    tokens=[i for i in tokens if i not in russian_stop_words]
  tokens=[snowball.stem(i) for i in tokens]
  return tokens


In [109]:
#проверим на первом примере как работает функция
tokenize_sentence(sentence_example)

['хохл',
 'эт',
 'отдушин',
 'затюкан',
 'россиянин',
 'мол',
 'вон',
 'хохл',
 'хуж',
 'есл',
 'хохл',
 'кисел',
 'придума']

**После того как готов массив токенов, можно применять tf-idf векторизацию. Это популярный способ для кодирования последовательности токенов, не используя эмбеддинги.
Конечно, у эмбеддингов большое количество преимуществ и это более мощный инструмент.**

In [110]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
vectorizer=TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x))
#векторизатор готов, можно его обучать

train_df, test_df = train_test_split(merged_df, test_size=0.3)
features=vectorizer.fit_transform(train_df['text'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [111]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
#создаем пайплайн
model_pipeline=Pipeline([
    ('vectorizer', TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x))),
    ('model', LogisticRegression(random_state=0))
])
#теперь пайплайн можно обучить

In [112]:
train_df

,text,label
63977,"иногда тебя просто терпят от скуки, от безысхо...",0
109356,нормальные украинцы живут не тужат - теперь об...,0
85325,тварь конченая сука двух стульная а не призидент,1
146833,"это просто фашисты,а не полиция они что думают...",1
8393,"Ну блдь, импеллер жеж, импеллер.\n",1
...,...,...
260743,"как мы скучаем,дочь постоянно спрашивает.кто т...",0
130093,как нам без них тяжело 🙏🙏🙏🙏🙏,0
160224,"песочница двухместная. если все сползуться,буд...",0
49851,1850доставка бекер метрого чейин,0


In [113]:
model_pipeline.fit(train_df['text'],train_df['label'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function <lambda> at 0x7a8e9c655fc0>)),
                ('model', LogisticRegression(random_state=0))])

In [114]:
model_pipeline.predict([''])

array([0])

In [115]:
#оценим качество модели
from sklearn.metrics import precision_score, recall_score, precision_recall_curve
precision_score(y_true=test_df['label'], y_pred=model_pipeline.predict(test_df['text']))

0.947296489279918

In [116]:
recall_score(y_true=test_df['label'], y_pred=model_pipeline.predict(test_df['text']))

0.7513041121875211

In [117]:
#используем GridSearchCV для поиска лучших гиперпараметров
from sklearn.model_selection import GridSearchCV
# создаем новый пайплайн
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x))),
    ('model', LogisticRegression(random_state=0))
])

# определяем сетку гиперпараметров
param_grid = {
    'model__C': [0.01, 0.1, 1, 10],  # регуляризация
    'model__solver': ['liblinear', 'lbfgs']  # методы оптимизации
}

# запускаем GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='precision', verbose=1)
#после выполнения grid_search.fit(...), лучшие параметры будут сохранены в grid_search.best_params_.
grid_search.fit(train_df['text'], train_df['label'])

print("Лучшие гиперпараметры: ", grid_search.best_params_)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extr

Лучшие гиперпараметры:  {'model__C': 0.01, 'model__solver': 'liblinear'}


In [118]:
# получаем предсказания для тестовой выборки
predictions = grid_search.predict(test_df['text'])

# оцениваем качество модели
precision = precision_score(test_df['label'], predictions)
recall = recall_score(test_df['label'], predictions)

print("Precision: ", precision)
print("Recall: ", recall)


Precision:  0.9990714948932219
Recall:  0.07289479032585869


In [119]:
#Такой высокий показатель precision и низкий показатель recall свидетельствует о том, что модель слишком "осторожна"
#и предсказывает токсичные комментарии очень редко


In [120]:
# создаем новый пайплайн с параметром class_weight='balanced', чтобы модель уделяла больше внимания меньшему классу
# так как датасет не сбалансирован
pipeline_with_better_recall = Pipeline([
    ('vectorizer', TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x))),
    ('model', LogisticRegression(random_state=0, class_weight='balanced'))  # сбалансируем веса классов
])

# определяем сетку гиперпараметров
param_grid_recall = {
    'model__C': [0.01, 0.1, 1, 10],  # регуляризация
    'model__solver': ['liblinear', 'lbfgs']  # методы оптимизации
}

# запускаем GridSearchCV для поиска лучших гиперпараметров
grid_search_recall = GridSearchCV(pipeline_with_better_recall, param_grid_recall, cv=3, scoring='recall', verbose=1)
grid_search_recall.fit(train_df['text'], train_df['label'])

# получаем предсказания вероятностей для тестовой выборки
probabilities = grid_search_recall.predict_proba(test_df['text'])[:, 1]

# устанавливаем более низкий порог, например 0.3 для повышения recall
new_threshold = 0.3
predictions_new = (probabilities >= new_threshold).astype(int)

# оцениваем качество модели
precision = precision_score(test_df['label'], predictions_new)
recall = recall_score(test_df['label'], predictions_new)


Fitting 3 folds for each of 8 candidates, totalling 24 fits


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extr

In [121]:
print(recall)
print(precision)

0.9132172617031367
0.7940154326441656


**Модель стала "смелее" указывать положительный класс. Это значит, что больше кол-во действительно токсичных комментариев классифицируются моделью как токсичные. Слегка возрос шанс ошибки (потому что precision стал меньше) и теперь с большей вероятностью модель может распознать нейтральный комментарий как токсичный.**

In [123]:
new_comments = ["Он просто туповат немного, забей", "А мне понравилось", "Иди отсюда козел."]
new_probabilities = grid_search_recall.predict_proba(new_comments)[:, 1]
# применим кастомный порог 0.3 для предсказания токсичности
new_predictions = (new_probabilities >= 0.3).astype(int)

for comment, prediction in zip(new_comments, new_predictions):
    print(f"Комментарий: {comment} | Токсичен: {'Да' if prediction == 1 else 'Нет'}")


Комментарий: Он просто туповат немного, забей | Токсичен: Да
Комментарий: А мне понравилось | Токсичен: Нет
Комментарий: Иди отсюда козел. | Токсичен: Да


In [124]:
#-------------------------------------------------------------------------------------------------------------------

In [125]:
def predict_toxicity(model, comments, threshold=0.3):
    """
    Предсказывает токсичность комментариев с помощью обученной модели.

    Параметры:
    - model: обученный пайплайн (GridSearchCV или Pipeline)
    - comments: строка или список строк (комментариев), которые нужно проверить
    - threshold: порог вероятности для определения токсичности (по умолчанию 0.3)

    Возвращает:
    - список с результатами предсказаний (1 - токсичный, 0 - нетоксичный)
    """
    if isinstance(comments, str):  # Если введена одна строка, преобразуем ее в список
        comments = [comments]

    # Преобразуем данные и делаем предсказание вероятностей
    probabilities = model.predict_proba(comments)[:, 1]

    # Применяем порог для получения бинарных предсказаний
    predictions = (probabilities >= threshold).astype(int)

    return predictions if len(predictions) > 1 else predictions[0]  # Возвращаем одно значение, если была одна строка


In [126]:
comment = "Не пойти бы тебе нафиг?"
result = predict_toxicity(grid_search_recall, comment)
print(f"Комментарий: {comment} | Токсичен: {'Да' if result == 1 else 'Нет'}")


Комментарий: Не пойти бы тебе нафиг? | Токсичен: Да
